In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

In [3]:
data = pd.read_csv("../data/cleaned_v1.csv")
data.head()

,Work Pressure,CGPA,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Work/Study Hours,Financial Stress,Depression,Gender_Female,Gender_Male,Age_Group,City_encoded,Student,Profession_encoded,suicidal_thoughts,family_history
0,5.0,-1.0,2.0,8.0,0.0,2.0,1.0,2.0,0,1.0,0.0,3,0.192690,0,0.048567,0,0
1,4.0,-1.0,3.0,5.0,2.0,2.0,7.0,3.0,1,0.0,1.0,0,0.142206,0,0.055649,1,0
2,5.0,9.0,2.0,6.0,0.0,2.0,3.0,1.0,1,0.0,1.0,1,0.159196,1,0.585061,1,0
3,5.0,-1.0,1.0,5.0,1.0,2.0,10.0,1.0,1,0.0,1.0,0,0.131293,0,0.055649,1,1
4,1.0,-1.0,1.0,6.0,2.0,2.0,9.0,4.0,0,1.0,0.0,1,0.125739,0,0.056610,1,1


In [4]:
## removing Gender_Male
data.drop(["Gender_Male"], axis=1, inplace=True)

In [5]:
y = data["Depression"]
X = data.drop(["Depression"], axis=1)

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [7]:
X_train.head()

,Work Pressure,CGPA,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Work/Study Hours,Financial Stress,Gender_Female,Age_Group,City_encoded,Student,Profession_encoded,suicidal_thoughts,family_history
33707,1.0,-1.0,5.0,8.0,2.0,2.0,4.0,2.0,0.0,4,0.175600,0,0.055649,0,1
133592,5.0,-1.0,4.0,5.0,0.0,3.0,11.0,4.0,0.0,4,0.242248,0,0.055649,0,1
113169,4.0,-1.0,3.0,8.0,1.0,3.0,3.0,1.0,0.0,3,0.214019,0,0.055649,0,1
42896,1.0,-1.0,1.0,8.0,2.0,2.0,8.0,5.0,0.0,0,0.144928,0,0.055649,1,0
82457,4.0,-1.0,5.0,6.0,1.0,1.0,0.0,2.0,0.0,2,0.134045,0,0.375086,0,0


In [8]:
def run_models(X_train, X_test, y_train, y_test):
    models = {
        "Logistic Regression": LogisticRegression(max_iter=1000),
        "Decision Tree": DecisionTreeClassifier(max_depth=5, random_state=42),
        "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
        "XGBoost": XGBClassifier(
            use_label_encoder=False, eval_metric="logloss", random_state=42
        ),
        "K-Nearest Neighbors": KNeighborsClassifier(n_neighbors=5),
        "Naive Bayes": GaussianNB(),
        "Dummy Classifier": DummyClassifier(strategy="most_frequent"),
    }

    results = {}

    for name, model in models.items():
        try:
            print(str("Currently training: " + name))
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            results[name] = accuracy
            print(str("Model " + name + f" has a accuracy of {accuracy}"))
        except Exception as e:
            results[name] = f"Error: {e}"

    return results

In [9]:
#results = run_models(X_train, X_test, y_train, y_test)
#print(results)

Hyperparameter tuning for:
- Logistic
- Random Forest
- XGBoost

In [10]:
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)
y_pred = log_reg.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.9355366027007818


In [11]:
xgboost = XGBClassifier(se_label_encoder=False, eval_metric="logloss", random_state=42)
xgboost.fit(X_train, y_train)
y_pred = xgboost.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.9357498223169864


In [12]:
random_for = RandomForestClassifier(n_estimators=100, random_state=42)
random_for.fit(X_train, y_train)
y_pred = random_for.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.9340796019900498


In [13]:
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix

def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else None
    
    print("Classification Report:")
    print(classification_report(y_test, y_pred))
    if y_prob is not None:
        print(f"ROC-AUC Score: {roc_auc_score(y_test, y_prob)}")
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    print(f"Accuracy score: {accuracy_score(y_test, y_pred)} ")


In [14]:
print("Evaluation report for Logistic Regression:\n")
evaluate_model(log_reg, X_test, y_test)

Evaluation report for Logistic Regression:

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.97      0.96     22986
           1       0.84      0.80      0.82      5154

    accuracy                           0.94     28140
   macro avg       0.90      0.88      0.89     28140
weighted avg       0.93      0.94      0.93     28140

ROC-AUC Score: 0.9738835732745625
Confusion Matrix:
[[22200   786]
 [ 1028  4126]]
Accuracy score: 0.9355366027007818 


In [15]:
print("Evaluation report for XGBoost:\n")
evaluate_model(xgboost, X_test, y_test)

Evaluation report for XGBoost:

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.96      0.96     22986
           1       0.83      0.81      0.82      5154

    accuracy                           0.94     28140
   macro avg       0.89      0.89      0.89     28140
weighted avg       0.94      0.94      0.94     28140

ROC-AUC Score: 0.9738361983493453
Confusion Matrix:
[[22135   851]
 [  957  4197]]
Accuracy score: 0.9357498223169864 


In [16]:
print("Evaluation report for Random Forest:\n")
evaluate_model(random_for, X_test, y_test)

Evaluation report for Random Forest:

Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.97      0.96     22986
           1       0.84      0.79      0.82      5154

    accuracy                           0.93     28140
   macro avg       0.90      0.88      0.89     28140
weighted avg       0.93      0.93      0.93     28140

ROC-AUC Score: 0.9714651139407257
Confusion Matrix:
[[22198   788]
 [ 1067  4087]]
Accuracy score: 0.9340796019900498 


Should try to reduce False Negatives, which is very high (close to 1000 for each model), since the data is imbalanced.

In [17]:
y_train.value_counts()

Depression
0    92147
1    20413
Name: count, dtype: int64

In [18]:
y_test.value_counts()

Depression
0    22986
1     5154
Name: count, dtype: int64

Tuning Logistic Regression

In [19]:
from scipy.stats import uniform, loguniform
from sklearn.model_selection import RandomizedSearchCV

# Define the parameter distribution
param_dist = {
    "C": loguniform(1e-4, 1e4),
    "penalty": ["l1", "l2", "elasticnet", "none"],
    "solver": ["lbfgs", "newton-cg", "liblinear", "sag", "saga"],
    "max_iter": [100, 1000, 2500, 5000],
}

# Create a RandomizedSearchCV object
random_search = RandomizedSearchCV(
    LogisticRegression(),
    param_distributions=param_dist,
    n_iter=100,  # Number of parameter settings that are sampled
    cv=5,
    random_state=42,
    n_jobs=-1,  # Use all available cores
)

In [20]:
#random_search.fit(X_train, y_train)

In [21]:
#best_params = random_search.best_params_
#best_model = random_search.best_estimator_

#accuracy = best_model.score(X_test, y_test)
#print(f"Best parameters: {best_params}")
#print(f"Accuracy: {accuracy:.3f}")

In [22]:
best_params = {
    "C": 3.6348739977385867,
    "max_iter": 5000,
    "penalty": "l2",
    "solver": "newton-cg",
}
log_reg_tuned = LogisticRegression(**best_params)
log_reg_tuned.fit(X_train, y_train)
evaluate_model(log_reg_tuned, X_test, y_test)

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.97      0.96     22986
           1       0.84      0.80      0.82      5154

    accuracy                           0.94     28140
   macro avg       0.90      0.88      0.89     28140
weighted avg       0.93      0.94      0.94     28140

ROC-AUC Score: 0.9738905286310667
Confusion Matrix:
[[22201   785]
 [ 1026  4128]]
Accuracy score: 0.9356432125088842 


Tuning XGBoost

In [26]:
param_dist = {
    "n_estimators": [100, 500, 900, 1100, 1500],
    "max_depth": [2, 3, 5, 10, 15],
    "learning_rate": [0.05, 0.1, 0.15, 0.20],
    "min_child_weight": [1, 2, 3, 4],
    "subsample": np.arange(0.5, 1.0, 0.1),
    "colsample_bytree": np.arange(0.5, 1.0, 0.1),
}
xgb_clf = XGBClassifier(objective="binary:logistic", tree_method="hist")

random_search = RandomizedSearchCV(
    estimator=xgb_clf,
    param_distributions=param_dist,
    n_iter=25,  # Number of parameter settings sampled
    scoring="accuracy",
    cv=5,  # Number of cross-validation folds
    verbose=1,
    n_jobs=-1,  # Use all available cores
    random_state=42,
)
random_search.fit(X_train, y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits


RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate...
                                           num_parallel_tree=None,
                                           random_state=None, ...),
                   n_iter=25, n_jobs=-1,
                   param_distributions={'colsample_bytree': array([0.5, 0.6, 0.7, 0.8, 0.9]),
                                        'learning_rate': [0.05, 0.1, 0.15, 0.2],
                                        'max_depth': [2, 3, 5, 10, 15],
                                        'min_child_weight': [1, 2, 3, 4],
                                        'n_estimators': [100, 500, 900, 1100,
                                                         1500],
                                        'subsample': array([0.5, 0.6, 0.7, 0.8, 0.9])},
                   random_state=42, scoring='accuracy', verbose=1)

In [27]:
best_params = random_search.best_params_
best_model = random_search.best_estimator_

In [29]:
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Best parameters: {best_params}")
print(f"Accuracy on test set: {accuracy}")

Best parameters: {'subsample': 0.5, 'n_estimators': 500, 'min_child_weight': 3, 'max_depth': 2, 'learning_rate': 0.1, 'colsample_bytree': 0.7}
Accuracy on test set: 0.9382373845060412


In [30]:
best_params = {
    "subsample": 0.5,
    "n_estimators": 500,
    "min_child_weight": 3,
    "max_depth": 2,
    "learning_rate": 0.1,
    "colsample_bytree": 0.7,
}
xgboost_tuned = XGBClassifier(
    objective="binary:logistic", tree_method="hist", **best_params
)
xgboost_tuned.fit(X_train, y_train)
evaluate_model(xgboost_tuned, X_test, y_test)

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.97      0.96     22986
           1       0.85      0.81      0.83      5154

    accuracy                           0.94     28140
   macro avg       0.90      0.89      0.90     28140
weighted avg       0.94      0.94      0.94     28140

ROC-AUC Score: 0.9758697707072189
Confusion Matrix:
[[22221   765]
 [  973  4181]]
Accuracy score: 0.9382373845060412 
